### Imports et initialisations

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import random

In [14]:
#raw_dataset_file = "clusters/1M_Std_clusters/som_400/som_400_cluster_349_raw_msg.txt"
raw_dataset_file = "datasets/haine2022_oversampling_africain.txt"

### Chargement des données

In [25]:
with open(raw_dataset_file, "r") as file:
    raw_dataset = file.read().splitlines()

In [26]:
sample_size = 30

idx_list = random.sample(range(len(raw_dataset)), sample_size)
raw_dataset = [raw_dataset[i] for i in idx_list]

### Chargement du modèle et classification des données

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-french")
#model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-french")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("models/dehatebert_tokenizer")
model = AutoModelForSequenceClassification.from_pretrained("models/dehatebert_model")

In [30]:
%%time
#en cas de pb de mémoire
predicted_classes_id = []
step = 10
max = 30
for i in range(0, max, step):
    inp = tokenizer(raw_dataset[i:i+step], return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        logits = model(**inp).logits
    predicted_classes_id += [elt.argmax().item() for elt in logits]

CPU times: user 7.34 s, sys: 5.81 ms, total: 7.35 s
Wall time: 1.84 s


In [27]:
%%time
predicted_classes_id = []
inp = tokenizer(raw_dataset, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    logits = model(**inp).logits
predicted_classes_id += [elt.argmax().item() for elt in logits]

CPU times: user 8.45 s, sys: 6.34 ms, total: 8.46 s
Wall time: 2.17 s


### Regroupement des messages par classe

In [33]:
n = 0
haineux = []
non_haineux = []
for i, msg in enumerate(raw_dataset):#[:max]):
    if (predicted_classes_id[i] == 1):
        haineux.append(msg)
        n += 1
    else :
        non_haineux.append(msg)
print("{} messages haineux sur {}".format(n, len(raw_dataset)))

11 messages haineux sur 30


#### Les messages classifiés comme haineux
- 2 messages litigieux
- 5 faux positifs
- 4 messages réellement haineux

In [34]:
for h in haineux: print(h)

Raf sale noir
Bah le babouin
Mdrr j’en ai lu des choses d’une hypocrisie totale mais la Arabe de Service n’a ps été dis pour moquer le fait d’être arabe, mais bien par l’utilisation de son origine comme justification d’un racisme inexistant dans la police, en gros « j’ai un Ami noir je suis ps raciste »
purée ce sale noir de Kevin mtn, y en a MARRE
T as oublié le pouce de nègre frère
Mais les #habitants de #SaintYrieix en #Limousin s'appellent toujours les cuou negre, les culs noirs ! comme leurs cochons #occitan #vocabulaire #Rabelais disait que c'étaient des mâche-raves ! cuou negre c'est plus chic ! #Limousin
Vous respectez personnes et vous ça vous fait buguer totale qu'une personne soit objective, arabes ou noire. C'est vous les raciste finalement. Et vous arrivez encore a renversé la situation, comparé Linda kebab au negre de maison... Vous êtes ridicule.
"C'eeeest noir !! Tu sais même pas laver un sol espèce de sale zinzin !!" (on te dit ça)Et toi tu frottes ce connard de sol to

#### Les messages classifiés comme non haineux
- deux messages litigieux
- le reste réellement non haineux

In [36]:
for n in non_haineux: print(n)

Des enfants sur Twitter ?
Caucasien ou négroïde?
je vais m’occuper de son cas
Ça fait des décennies que plus personne n'emploie le mot nègre dans la vie mais quand ça devient la norme dans la littérature c'est de la faute du wokisme. Heureusement qu'on peut encore dire gros cons au
Jake Lamar: «Eliminer le mot “nègre” des œuvres de William Faulkner n’aurait aucun sens!»https://t.co/7z0BfW2n6h
Insupportable ces publicités qui prônent le métissage. Et dans 95% des cas c'est un congoïde avec une caucasoïde
l'origine de la personne mais sa fonction dans le groupe : la caution morale d'un groupe problématique dans son fonctionnement, au même titre qu'on parle de nègre pour parler d'un auteur qui écrit pour quelqu'un d'autre. Il n'est évidemment pas forcément noir.
comme « Xbox has no games » je te propose de te racheter cette horrible monolithe noir aux couleurs de cet horrible jeu qu’est halo… ok? C’est un bon service rendu de ma part.
Quand on ne connaît on se tait et on demande.Prendre l

In [21]:
model.connon_haineux.id2label[1]

'HATE'